In [8]:
import requests
import pandas as pd

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash' ] 
index = []
data = []
btc_exchange_rate = 0

def get_exchange_rate(tag, price):
    if tag == 'BTC':
        global btc_exchange_rate
        btc_exchange_rate = price
        return 1
    return price

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  get_exchange_rate(c['tag'], c['exchanges'][0]['price']), c['algorithm'], c['nethash'] ])

global_df = pd.DataFrame(data, index=index, columns=columns)
global_df['blocks_per_day'] = 86400 / global_df.block_time
global_df['block_reward_btc'] = global_df.block_reward * global_df.exchange_rate_btc
global_df['daily_emmisions_btc'] = global_df.blocks_per_day * global_df.block_reward_btc
global_df.loc['DOGE+LTC', ['daily_emmisions_btc']] = global_df.loc['DOGE', ['daily_emmisions_btc']] + global_df.loc['LTC', ['daily_emmisions_btc']]
print('Global blockaing data:')
# print(df)
global_df

Global blockaing data:


,block_time,block_reward,exchange_rate_btc,algorithm,nethash,blocks_per_day,block_reward_btc,daily_emmisions_btc
BCH,485.0000,3.125000,3.403288e-03,SHA-256,3505717486588960824,178.144330,0.010635,1.894614
BTC,584.0000,3.159799,1.000000e+00,SHA-256,839631325343408904109,147.945205,3.159799,467.477100
DOGE,63.0000,10000.000000,2.572000e-06,Scrypt,1389946861916890,1371.428571,0.025720,35.273143
KAS,1.0002,65.548390,9.142159e-07,kHeavyHash,1314864663863720255,86382.723455,0.000060,5.176517
LTC,148.0000,6.250000,1.190520e-03,Scrypt,2213508382450659,583.783784,0.007441,4.343789
RVN,60.0000,2500.000000,1.431520e-07,KawPow,6684954192481,1440.000000,0.000358,0.515347
DOGE+LTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.616932


In [9]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
market_df = pd.DataFrame(index=index)
market_df['hashrate'] = algo_data_df['speed'] * factors_data_df['mining_factor'] / factors_data_df['market_factor']
market_df['paying'] = algo_data_df['paying'] * factors_data_df['price_factor'] / sats2btc
market_df['daily_consumption_btc'] = market_df.hashrate * market_df.paying
print('Nicehash marketplace data:')
# print(market_data_df)
market_df

Nicehash marketplace data:


,hashrate,paying,daily_consumption_btc
SCRYPT,93.639669,0.022033,2.063203
SHA256ASICBOOST,7.135472,0.552080,3.939351
KAWPOW,0.765165,0.081542,0.062393
KHEAVYHASH,54.364986,0.003894,0.211671


In [12]:
min_order_amount_btc = 5e-3
max_market_utilisation = 0.76

limits_df = pd.DataFrame(index=market_df.index)
limits_df['max_daily_consumption_btc'] = market_df.daily_consumption_btc * max_market_utilisation
limits_df['max_hourly_consumption_btc'] = limits_df.max_daily_consumption_btc / 24
limits_df['max_hourly_consumption_dollar'] = round(limits_df.max_hourly_consumption_btc * btc_exchange_rate, 2)
limits_df

,max_daily_consumption_btc,max_hourly_consumption_btc,max_hourly_consumption_dollar
SCRYPT,1.568034,0.065335,6384.14
SHA256ASICBOOST,2.993906,0.124746,12189.48
KAWPOW,0.047419,0.001976,193.06
KHEAVYHASH,0.160870,0.006703,654.97
